<a href="https://colab.research.google.com/github/burn0/Deepnoid-Education/blob/main/LSGAN_keras_bykim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Source code :**

J. Brownlee, "How to Develop a Least Squares Generative Adversarial Network (LSGAN) in Keras," Machine Learning Mastery, 2019

https://machinelearningmastery.com/least-squares-generative-adversarial-network/

# **Purpose**

LSGAN을 이용하여 MNIST 데이터 (숫자 손글씨)를 생성해봅시다.

# **1. Module import**
코드 실행에 필요한 module을 import 합니다.

만약, 본인이 코드를 수정할 때 필요로 하는 모듈이 있다면 추가해주시면 됩니다.

In [ ]:
# example of lsgan for mnist
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import Activation
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.initializers import RandomNormal
from matplotlib import pyplot

# **2. Discriminator model**

Discriminator으로 사용할 network를 구성해봅시다.

2개의 convolution layer와 1개의 fully-connected layer로 구성하였습니다.

만약 본인이 원하는 구성으로 바꾸고 싶다면 해당코드 파일을 복사하신 후에 본인의 colab 환경에서 바꾸셔도 좋습니다.

Loss는 mse(mean squared error) funcation을 사용합니다.

In [ ]:
# define the standalone discriminator model
def define_discriminator(in_shape=(28,28,1)):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# define model
	model = Sequential()
	# downsample to 14x14
	model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init, input_shape=in_shape))
	model.add(BatchNormalization())
	model.add(LeakyReLU(alpha=0.2))
	# downsample to 7x7
	model.add(Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
	model.add(BatchNormalization())
	model.add(LeakyReLU(alpha=0.2))
	# classifier
	model.add(Flatten())
	model.add(Dense(1, activation='linear', kernel_initializer=init))
	# compile model with L2 loss
	model.compile(loss='mse', optimizer=Adam(lr=0.0002, beta_1=0.5))
	return model

# **3. Generator model**

Generator로 사용할 network를 구성해봅시다.

1차원 input z값으로 부터 2차원 image를 생성해야하기 때문에 reshape 이후에 Conv2DTranpose 내장함수를 이용하여 영상을 확장해줍니다.

In [ ]:
# define the standalone generator model
def define_generator(latent_dim):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# define model
	model = Sequential()
	# foundation for 7x7 image
	n_nodes = 256 * 7 * 7
	model.add(Dense(n_nodes, kernel_initializer=init, input_dim=latent_dim))
	model.add(BatchNormalization())
	model.add(Activation('relu'))
	model.add(Reshape((7, 7, 256)))
	# upsample to 14x14
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
	model.add(BatchNormalization())
	model.add(Activation('relu'))
	# upsample to 28x28
	model.add(Conv2DTranspose(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
	model.add(BatchNormalization())
	model.add(Activation('relu'))
	# output 28x28x1
	model.add(Conv2D(1, (7,7), padding='same', kernel_initializer=init))
	model.add(Activation('tanh'))
	return model

# **4. GAN model**

미리 구성한 Discriminator와 Generator를 이용하여 GAN 모델을 구성합니다.

GAN 모델에서는 Generator만 학습되어야 하기 때문에 discriminator가 학습되지 않도록 설정해줍니다.

In [ ]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
	# make weights in the discriminator not trainable
	for layer in discriminator.layers:
		if not isinstance(layer, BatchNormalization):
			layer.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(generator)
	# add the discriminator
	model.add(discriminator)
	# compile model with L2 loss
	model.compile(loss='mse', optimizer=Adam(lr=0.0002, beta_1=0.5))
	return model

# **5. Data prepare**

학습에 사용할 MNIST dataset을 불러오고 Normalization 시켜줍니다.
학습에 이용할 수 있게끔 해주는 function을 define 합니다.

In [ ]:
# load mnist images
def load_real_samples():
	# load dataset
	(trainX, _), (_, _) = load_data()
	# expand to 3d, e.g. add channels
	X = expand_dims(trainX, axis=-1)
	# convert from ints to floats
	X = X.astype('float32')
	# scale from [0,255] to [-1,1]
	X = (X - 127.5) / 127.5
	return X

In [ ]:
# # select real samples
def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# select images
	X = dataset[ix]
	# generate class labels
	y = ones((n_samples, 1))
	return X, y

In [ ]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

In [ ]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = generator.predict(x_input)
	# create class labels
	y = zeros((n_samples, 1))
	return X, y

In [ ]:
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, latent_dim, n_samples=100):
	# prepare fake examples
	X, _ = generate_fake_samples(g_model, latent_dim, n_samples)
	# scale from [-1,1] to [0,1]
	X = (X + 1) / 2.0
	# plot images
	for i in range(10 * 10):
		# define subplot
		pyplot.subplot(10, 10, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(X[i, :, :, 0], cmap='gray_r')
	# save plot to file
	filename1 = 'generated_plot_%06d.png' % (step+1)
	pyplot.savefig(filename1)
	pyplot.close()
	# save the generator model
	filename2 = 'model_%06d.h5' % (step+1)
	g_model.save(filename2)
	print('Saved %s and %s' % (filename1, filename2))

In [ ]:
# create a line plot of loss for the gan and save to file
def plot_history(d1_hist, d2_hist, g_hist):
	pyplot.plot(d1_hist, label='dloss1')
	pyplot.plot(d2_hist, label='dloss2')
	pyplot.plot(g_hist, label='gloss')
	pyplot.legend()
	filename = 'plot_line_plot_loss.png'
	pyplot.savefig(filename)
	pyplot.close()
	print('Saved %s' % (filename))

# **6. Training**

학습의 순서는 아래와 같습니다.

1. Discriminator 학습

2. GAN 모델을 이용한 Generator 학습

In [ ]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=20, n_batch=64):
	# calculate the number of batches per training epoch
	bat_per_epo = int(dataset.shape[0] / n_batch)
	# calculate the number of training iterations
	n_steps = bat_per_epo * n_epochs
	# calculate the size of half a batch of samples
	half_batch = int(n_batch / 2)
	# lists for storing loss, for plotting later
	d1_hist, d2_hist, g_hist = list(), list(), list()
	# manually enumerate epochs
	for i in range(n_steps):
		# prepare real and fake samples
		X_real, y_real = generate_real_samples(dataset, half_batch)
		X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
		# update discriminator model
		d_loss1 = d_model.train_on_batch(X_real, y_real)
		d_loss2 = d_model.train_on_batch(X_fake, y_fake)
		# update the generator via the discriminator's error
		z_input = generate_latent_points(latent_dim, n_batch)
		y_real2 = ones((n_batch, 1))
		g_loss = gan_model.train_on_batch(z_input, y_real2)
		# summarize loss on this batch
		print('>%d, d1=%.3f, d2=%.3f g=%.3f' % (i+1, d_loss1, d_loss2, g_loss))
		# record history
		d1_hist.append(d_loss1)
		d2_hist.append(d_loss2)
		g_hist.append(g_loss)
		# evaluate the model performance every 'epoch'
		if (i+1) % (bat_per_epo * 1) == 0:
			summarize_performance(i, g_model, latent_dim)
	# create line plot of training history
	plot_history(d1_hist, d2_hist, g_hist)

In [ ]:
# size of the latent space
latent_dim = 100

In [ ]:
# create the discriminator
discriminator = define_discriminator()

In [ ]:
# create the generator
generator = define_generator(latent_dim)

In [ ]:
# create the gan
gan_model = define_gan(generator, discriminator)

In [ ]:
# load image data
dataset = load_real_samples()
print(dataset.shape)

11493376/11490434 [==============================] - 0s 0us/step
(60000, 28, 28, 1)


In [ ]:
# train model
train(generator, discriminator, gan_model, dataset, latent_dim)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
>13748, d1=0.283, d2=0.295 g=0.279
>13749, d1=0.278, d2=0.292 g=0.289
>13750, d1=0.289, d2=0.290 g=0.280
>13751, d1=0.297, d2=0.291 g=0.279
>13752, d1=0.282, d2=0.292 g=0.285
>13753, d1=0.279, d2=0.285 g=0.287
>13754, d1=0.282, d2=0.283 g=0.281
>13755, d1=0.298, d2=0.272 g=0.279
>13756, d1=0.284, d2=0.287 g=0.274
>13757, d1=0.297, d2=0.294 g=0.278
>13758, d1=0.273, d2=0.302 g=0.282
>13759, d1=0.275, d2=0.287 g=0.282
>13760, d1=0.277, d2=0.301 g=0.279
>13761, d1=0.281, d2=0.281 g=0.285
>13762, d1=0.287, d2=0.280 g=0.281
>13763, d1=0.282, d2=0.288 g=0.278
>13764, d1=0.278, d2=0.282 g=0.276
>13765, d1=0.291, d2=0.321 g=0.280
>13766, d1=0.292, d2=0.279 g=0.278
>13767, d1=0.278, d2=0.289 g=0.281
>13768, d1=0.282, d2=0.299 g=0.282
>13769, d1=0.281, d2=0.295 g=0.286
>13770, d1=0.284, d2=0.284 g=0.276
>13771, d1=0.285, d2=0.275 g=0.275
>13772, d1=0.285, d2=0.298 g=0.276
>13773, d1=0.281, d2=0.286 g=0.277
>13774, d1=0.288, d2=0.295 g=0.280
>13775, d1=0.280, d